# Kundenbewertungen - Gut vs. Schlecht (in Python)

Die Meinungen von Kunden sind für jedes Unternehmen ein zentraler Punkt, um das angebotene Produkt zu verbessern. Einerseits will man zufriedene Kunden, um diese zu halten, andererseits will man auch, dass die bisherigen Kunden Empfehlungen in ihrem sozialen Feld aussprechen. Von dieser Warte ist die Analyse von Kundenbewertungen von enormer Wichtigkeit.

Für viele Bereiche gibt es speziell eingerichtete Plattformen, um die Kundenzufriedenheit zu messen: Dies reicht von der Vergabe von Sternchen nach einem Kauf auf Ebay bis hin zu ganz individuellen Kommentaren, die (online) von anderen potentiellen Kunden eingesehen wird. Ich persönlich schaue mir immer die *Google Reviews* eines Restaurants an, bevor ich dort das erste Mal essen gehe. Sind die Bewertungen nicht gut genug, suche ich mir ein anderes Lokal.

Dazu passend schauen wir uns heute an, wie man ein sogenanntes *Bag of Words* aus Kundenempfehlungen erstellt, um diese dann mittels eines Klassifizierungsmodells (hier: Random Forest) in *negativ* und *positiv* zu unterteilen. Der Nutzen für unternehmen besteht hierbei darin, dass zunächst ein grober Überblick geschaffen wird - kommt das Produkt bzw. der Service an oder läuft etwas völlig verkehrt? Im nächsten Schritt kann man sich dann negativen Bewerungen widmen und versuchen, gegebene Kritik umzusetzen.

Zunächst laden wir aber wieder einmal die zwei Standard-Bibliotheken *numpy* und *pandas*. *matplotlib* brauchen wir dieses Mal nicht.

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd

## Datensatz

Wieder einmal finden wir passende Datensätze auf Kaggle.com. Bedenkt immer, wenn ihr ein Portfolio (wie ich gerade) aufbaut, dass die Daten, die ihr verwendet, nicht in Konflikt mit Rechten Dritter liegen. Ein weiterer Vorteil von Kaggle-Datensätzen: Sie sind der breiten Masse zugänglich und jeder, der die hier vorgestellten Techniken ausprobieren will, kann dies mit genau den gleichen Daten tun.

Nun laden wir aber einmal unser Datenset *Restaurant_Reviews.tsv*. Wie man an der Endung bereits erkennt, handelt es sich um *tap separated values*, also Einträge, die durch einen Tab von einander getrennt sind im Gegensatz zu *csv* (*comma separated values*).

Zum Einlesen mittels Pandas müssen wir hierzu den korrketen "Delimiter" bestimmen. Standardmäßig ist das das Komma, hier ist es der Tab bzw. in Notation *\t'.

Ein weiterer Punkt sind Anführungszeichen, die uns hier Probleme machen könnten. Diese kann man mittels der Option *quoting = 3* ignorieren. Dann 'mal los.

In [2]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


Wie wir sehen, besteht unser Datenset aus zwei Spalten: dem eigentlich (schriftlichen) Review und dem Tenor des Reviews - also ob der Kunde mit seinem Review sagen will, dass es ihm gefallen hat oder eben nicht.

So weit, so gut. Als nächstes bringen wir nun die Reviews in ein Format (das sogenannte *Bag of Words* Format), mit dem wir unser Modell später füttern.

## Bag of Words

*Bag of Words*, wörtlich ein "Beutel von Wörtern" ist eine lange Liste mit aneinander gereiten Wörtern. Natürlich tragen Wörter selbst Bedeutung, aber auch der Kontext, also unter Anderem die umgebenden Wörter sind wichtig.

### Cleaning Text

Um einen guten Beutel zu bauen, werden wir nun die verschiedenen Wörter der Reviews in ein allgemeines Format bringen. Hierzu nutzen wir zunächst *Regular Expressions* mit der entsprechenden Bibliothek (*re*):

In [3]:
import re

Zunächst stellen wir einmal fest, dass Satzzeichen weitestgehend unwichtig für unsere Analyse sind. Also entfernen wir diese bzw. behalten nur Buchstaben. Alles andere ersetzen wir mit einem Leerzeichen. Genau für diesen Zweck gibt es die *substitute*-Funktion bei den *Regular Expressions*, *re.sub()*. Dieser füttert man ein Muster, welches ersetzt, anschließend das Muster, welches stattdessen benutzt werden soll. Zum Schluss gibt man noch den Input-String an.

In unserem Fall sieht das so aus:

In [4]:
review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][0])
review

'Wow    Loved this place '

Aber Moment 'mal! Wir wollen doch Satzzeichen entfernen und nicht Buchstaben... und das haben wir auch nicht. Mit Hilfe von eckigen Klammern ([ & ]) kann man eine Gruppe von Symbolen angeben; und wenn man dieser Gruppe ein "^" voranstellt, wird die Gruppe aus den komplementären Symbolen erstellt. Für uns heißt das, dass wir nicht nur Satzzeichen, sondern auch Zahlen entfernt haben. Aber das ist erstmal in Ordnung.

Als nächstes ist Groß- und Kleinschreibung dran. Und die ist für unseren Wörtersack auch egal. Also machen wir alle Buchstaben klein:

In [5]:
review = review.lower()
review

'wow    loved this place '

Jetzt wird es noch einmal besonders interessant. Im obigen Beispiel kommt das Wort "loved" vor, ein starkes Wort. Aber unterscheidet es sich von den verschiedenen Konjugationen bzw. Deklinationen? Nicht wirklich. Was zählt ist der Wortstamm. Und um diesen zu isolieren, nutzen wir das *Natural Language Toolkit* (*nltk*) und den sogenannten *PorterStemmer*.

Wenn man darüber nachdenkt, gibt es aber auch solche Worte, die selbst keine Bedeutung tragen, sondern nur verschiedene Satzteile miteinander verbinden. Im Beispiel wäre dies "this". Solche Wörter nennt man auch *stopwords* und wir brauchen auch sie nicht in unserer Bag, also entfernen wir sie gleich mit.

Hier die entsprechenden Bibliotheken:

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /home/george/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Wichtig, bevor wir unser Review filtern, ist das Format: Später wollen wir die wichtigen Wörter eines Reviews aneinandergereit haben, doch für das *Natural Language Toolkit* nehmen wir uns die Wörter einzelnd vor. Von daher müssen wir unser Review noch einmal zerlegen bevor wir es später wieder zusammensetzen:

In [7]:
review = review.split()
review

['wow', 'loved', 'this', 'place']

Nun kommen wir zur Isolation der einzelnen Wortstämme sowie dem Entfernen der *stopwords*. Der folgende Befehl sieht etwas kompliziert aus, aber ich werde Schritt für Schritt erklären, was wir tun:

In [8]:
ps = PorterStemmer()
review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
review

['wow', 'love', 'place']

Was wir hier haben ist eine *for*-Schleife über alle Wörter in unserem Review. Statt die Wörter aber wieder zurück in das Review zu stecken, stecken wir nur den Wortstamm zurück. Die erreichen wir mittels *ps.stem(word)*. Anschließend fügen wir aber noch eine Bedingung hinzu: Das Wort darf sich nicht in unserer Liste der *stopwords* befinden. Hier wählen wir konkret englische *stopwords* aus.

Ein Detail noch: Die *set*-Funktion, die wir auf unsere *stopwords* anwenden, braucht man strenggenommen nicht. Durch *set* wird aus der Liste von *stopwords* jedoch eine geordnete Sequenz, durch welche unsere Schleife schneller durchlaufen kann. Das könnte bei größeren Reviews also durchaus relevant werden.

Zuletzt fügen wir jetzt noch die einzelnen Wörter wieder zusammen:

In [9]:
review = ' '.join(review)
review

'wow love place'

### Bag Creation

Das Ganze haben wir gerade beispielhaft für das erste Review gemacht. Jetzt geht es daran, dies auch für die anderen zu tun. Hierfür nutzen wir ebenfalls eine *for*-Schleife und fügen die einzelnen Reviews schließlich unserer *Corpus*-Liste, also unseren Wörterbeutel hinzu.

In [10]:
# Creating the Corpus
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

Unser Wörterbeutel enthält nun alle Reviews, nachdem sie für unsere Zwecke "gereinigt" wurden - also keine Satzzeichen, Zahlen, einheitliche Kleinschreibung sowie keine Bindewörter. Nun bereiten wir unseren Beutel noch für die letzte Auswertung fertig. Hierzu erstellen wir eine *sparse matrix* aus allen Wörtern, die uns einfach zeigt, welche Wörter mit welchen anderen Wörtern auftauchen.

Zur Erstellung der *sparse matrix* nutzen wir den *CountVectorizer* aus der Scikit-Learn-Bibliothek *sklearn.feature_extraction_text*. Diesen wenden wir unseren Corpus an und transformieren das ganze noch in ein Grid mittels des *.toarray()*-Befehls. Diese Grid (X) enthält dann alle unabhängigen Variablen, die unserem Modell als Input dient. Wie gewohnt fassen wir die dazugehörige abhängige Variable in y zusammen:

In [11]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

Und das war es auch schon! Weiter geht es mit unserem Modell.

## Modell - Random Forest

Für unser Datenset hier werde ich nun das *Random Forest*-Modell anwenden. Ich werde hier nicht weiter auf das Modell an sich eingehen, es sei nur gesagt, dass es zur Klassifierung genutzt werden kann. Da es uns hier nicht primär um Genauigkeit geht, hätten wir also auch jedes andere Klassifizierungsmodell nutzen können.

Zunächst teilen wir unseren Datensatz aber erst einmal in ein Trainings- und ein Testset. Ich wähle hier wieder ein Verhältnis von 80:20 von Trainings- zu Testdaten und setze den *random_state = 0*, damit ihr eure Ergebnisse mit meinen vergleichen könnt.

In [12]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Als nächstes kommt *Feature Scaling*, welches wir mittels des *StandardScaler*s der *sklearn.preprocessing*-Bibliothek implementieren.

In [13]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Was euch aufgefallen sein wird, ist dass wir für unser Trainingsset *X_train* und unser Testset *X_test* zwei unterschiedliche Methoden verwendet haben. Dies hat einen wichtigen Grund: Beim Feature Scaling ist es von Vorteil, die Daten um ihren Erwartungswert herum zu zentrieren. Konkret braucht man beim Zentrieren neben dem Erwartungswert (typischerweise angegeben als $\mu$) noch die Standardabweichung $\sigma$. Mit der *.fit*-Methode können beide Werte ermittelt werden. *.fit_transform* ermittelt hierbei erst die Werte und nutzt diese dann, um die Daten zu transformieren.

Nun, zur korrekten Auswertung müssen $\mu$ und $\sigma$ des Trainingssets aber die gleichen im Testset sein. Aus diesem Grund wenden wir nicht erneut die *.fit*-Methode auf das Testset an, sondern transformieren ausschließlich. Die dabei verwendeten Werte sind die des zuletzt durchgeführten Fits.

Und jetzt müssen wir nur noch unser Modell trainieren. Hierzu laden wir den *RandomForestClassifier* aus der Bibliothek *sklearn.ensemble* und wenden ihn auf unsere Trainingsdaten an:

In [14]:
# Fitting Random Forest classifier to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10,
                                    criterion = 'entropy',
                                    random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Auswertung

Mittels der *.predict()*-Funktion des Classifiers können wir nun das trainierte Modell auf unser Testset anwenden.

In [15]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

Um nun die Güte unseres Modells zu bewerten, stehen uns verschiedene Wege offen: Genauigkeit, Präzision, aber auch Größen wie *Recall* oder die sogenannte *F1 Score* können herangezogen werden. Eines haben diese Werte alle gemein: Sie leiten sich aus der *Confusion Matrix* unseres Modells ab. Wie genau, das könnt ihr aus dem Code-Abschnitt unten entnehmen.

In [16]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

accuracy = ( cm[0, 0] + cm[1, 1] ) / y_test.size
precision = cm[1, 1] / ( cm[1, 1] + cm[0, 1] )
recall = cm[1, 1] / ( cm[1, 1] + cm[1, 0] )
f1_score = 2 * precision * recall / (precision + recall)

print(cm)
print('Accuracy:  ', accuracy)
print('Precision: ', precision)
print('Recall:    ', recall)
print('F1 Score:  ', f1_score)

[[87 10]
 [46 57]]
Accuracy:   0.72
Precision:  0.8507462686567164
Recall:     0.5533980582524272
F1 Score:   0.6705882352941177


*Accuracy* 72%. Nun, das könnte deutlich besser sein, sollte uns an dieser Stelle aber nicht entmutigen. Dies war unser erster Durchlauf mit einem *Bag of Words* und die Hyperparameter unseres Random Forests haben wir noch gar nicht angefasst!
 
## Zusammenfassung

Rekapitulieren wir: Unser Ziel war es, aus einem Datensatz von Kundenbewertungen ein Modell zu entwickeln, das bestimmen kann, ob eine Bewertung gut oder schlecht ist. Es handelte sich also um ein klassisches Klassifizierungsproblem. Bevor wir aber zum Trainieren unseres Modells übergehen konnten, mussten wir die Bewertungen erst einmal in ein geeignetes Format bringen - hier spricht zu englisch von *Bag of Words*.

Hierfür haben wir erst einmal für die Bewertung irrelevante Zeichensetzung, sowie Zahlen entfernt. Groß- und Kleinschreibung war uns auch nicht wichtig, so dass wir alle Großbuchstaben in Kleinbuchstaben umgewandelt haben. Zuletzt löschten wir dann noch alle Füll- und Bindewörter mittels der englischen *stopwords*-Liste und reduzierten alle Wörter auf ihren Stamm, wofür wir den *PorterStemmer* verwendeten.

Zu guter Letzt erzeugten wir aus der *Bag of Words* eine *sparse matrix*, die wir anschließend dem Klassifizierer unseres Modells, hier einem Random Forest, fütterten. Ohne Tuning kamen wir dabei auf eine Genauigkeit von 72%. Ausbaufähig, aber schon ein guter Anfang!